# Exercice 1 : Charger les datasets et les envoyer sur Kafka

```bash
kafka-topics.sh --create --topic train_lines --bootstrap-server kafka:9092 --replication-factor 1 --partitions 1
kafka-topics.sh --create --topic train_users --bootstrap-server kafka:9092 --replication-factor 1 --partitions 1
```


In [17]:
from kafka import KafkaProducer
import json, time

producer = KafkaProducer(bootstrap_servers="kafka:9092",
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

lines = json.load(open("/work/data/train_lines.json"))
users = json.load(open("/work/data/train_users.json"))

for l in lines:
    producer.send("train_lines", value=l)
for u in users:
    producer.send("train_users", value=u)
producer.flush()

{'line_id': 1, 'line_name': 'IC-01', 'origin': 'Bruxelles-Midi', 'destination': 'Liège-Guillemins', 'distance_km': 98, 'avg_duration_min': 62}
{'line_id': 2, 'line_name': 'IC-02', 'origin': 'Bruxelles-Central', 'destination': 'Namur', 'distance_km': 64, 'avg_duration_min': 55}
{'line_id': 3, 'line_name': 'IC-03', 'origin': 'Anvers-Central', 'destination': 'Gand-Saint-Pierre', 'distance_km': 56, 'avg_duration_min': 49}
{'line_id': 4, 'line_name': 'IC-04', 'origin': 'Bruxelles-Nord', 'destination': 'Charleroi-Sud', 'distance_km': 61, 'avg_duration_min': 57}
{'line_id': 5, 'line_name': 'IC-05', 'origin': 'Liège-Guillemins', 'destination': 'Luxembourg', 'distance_km': 125, 'avg_duration_min': 115}
{'line_id': 6, 'line_name': 'IC-06', 'origin': 'Anvers-Central', 'destination': 'Bruges', 'distance_km': 93, 'avg_duration_min': 82}
{'line_id': 7, 'line_name': 'IC-07', 'origin': 'Bruxelles-Midi', 'destination': 'Mons', 'distance_km': 64, 'avg_duration_min': 52}
{'line_id': 8, 'line_name': 'IC-0

# Exercice 2 : Lecture des topics Kafka (streaming) et écriture en Bronze

In [3]:
# Cell: imports & SparkSession bootstrap
import os
import json
import time
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, from_json, to_json, struct, expr, lit, when, avg, count, desc, window, size, filter as spark_filter
from pyspark.sql.functions import udf

# Adjust these if you prefer to pass --packages in spark-submit
KAFKA_PACKAGES = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,org.postgresql:postgresql:42.6.0"

spark = SparkSession.builder \
    .appName("TrainPipelineNotebook") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print("Spark:", spark.version)


Spark: 3.4.1


In [19]:
# Cell: configs
KAFKA_BOOTSTRAP = "kafka:9092"   # ou "localhost:9092" selon ton setup
TOPIC_LINES = "train.lines"
TOPIC_USERS = "train.users"

BRONZE_LINES_PATH = "/tmp/bronze/lines"
BRONZE_USERS_PATH = "/tmp/bronze/users"
CHK_BRONZE_LINES = "/tmp/checkpoints/bronze_lines"
CHK_BRONZE_USERS = "/tmp/checkpoints/bronze_users"

SILVER_PATH = "/tmp/silver"
CHK_SILVER = "/tmp/checkpoints/silver"

POSTGRES_URL = "jdbc:postgresql://postgres:5432/events"
POSTGRES_USER = "pguser"
POSTGRES_PASSWORD = "pgpass"

DATA_DIR = "/work/data"   # endroit où se trouvent train_lines.json / train_users.json


In [4]:
df_lines_stream = (spark.readStream
                   .format("kafka")
                   .option("kafka.bootstrap.servers", "kafka:9092")
                   .option("subscribe", "train_lines")
                   .load())

df_users_stream = (spark.readStream
                   .format("kafka")
                   .option("kafka.bootstrap.servers", "kafka:9092")
                   .option("subscribe", "train_users")
                   .load())

(df_lines_stream
 .writeStream
 .format("parquet")
 .option("checkpointLocation", "./tmp/chk_lines")
 .option("path", "./tmp/bronze/train_lines")
 .outputMode("append")
 .start())

(df_users_stream
 .writeStream
 .format("parquet")
 .option("checkpointLocation", "./tmp/chk_users")
 .option("path", "./tmp/bronze/train_users")
 .outputMode("append")
 .start())


25/10/05 19:12:16 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/10/05 19:12:17 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


# Exercice 3 : Conversion en lecture statique (df_static)

In [5]:
bronze_lines = spark.read.parquet("./tmp/bronze/train_lines")
bronze_users = spark.read.parquet("./tmp/bronze/train_users")

from pyspark.sql.functions import col
bronze_lines = bronze_lines.selectExpr("CAST(value AS STRING) AS json")
bronze_users = bronze_users.selectExpr("CAST(value AS STRING) AS json")


# Exercice 4 : Parsing JSON vers colonnes structurées (Bronze → Silver)

In [43]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

# Schéma explicite pour les lignes de train
schema_lines = StructType([
    StructField("line_id", StringType(), False),
    StructField("line_name", StringType(), True),
    StructField("origin", StringType(), True),
    StructField("destination", StringType(), True),
    StructField("distance_km", StringType(), True),
    StructField("avg_duration_min", StringType(), True)
])

# Schéma explicite pour les trajets utilisateurs
schema_users = StructType([
    StructField("user_id", StringType(), False),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("line_id", StringType(), True),
    StructField("trip_date", DateType(), True),
    StructField("actual_duration_min", IntegerType(), True),
    StructField("delay_min", IntegerType(), True),
    StructField("satisfaction", DoubleType(), True)
])

from pyspark.sql.functions import from_json, col

silver_lines = bronze_lines.select(from_json(col("json"), schema_lines).alias("data")).select("data.*")
silver_users = bronze_users.select(from_json(col("json"), schema_users).alias("data")).select("data.*")

display(silver_lines.limit(5))
display(silver_users.limit(5))


DataFrame[line_id: string, line_name: string, origin: string, destination: string, distance_km: string, avg_duration_min: string]

DataFrame[user_id: string, name: string, age: int, line_id: string, trip_date: date, actual_duration_min: int, delay_min: int, satisfaction: double]

# Exercice 5 : Sauvegarde des Silver en DB

In [39]:
silver_lines.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres:5432/events") \
    .option("dbtable", "silver_train_lines") \
    .option("user", "pguser") \
    .option("password", "pgpass") \
    .mode("overwrite") \
    .save()


# Exercice 6 : Vérifier la cohérence des données

In [40]:
print(silver_lines.count(), "lignes de train")
print(silver_users.count(), "trajets utilisateur")
silver_users.filter(col("line_id").isNull()).show()


60 lignes de train
200 trajets utilisateur
+-------+----+---+-------+---------+-------------------+---------+------------+
|user_id|name|age|line_id|trip_date|actual_duration_min|delay_min|satisfaction|
+-------+----+---+-------+---------+-------------------+---------+------------+
+-------+----+---+-------+---------+-------------------+---------+------------+



# Exercice 7 : Nettoyer les données manquantes

In [41]:
silver_users = silver_users.na.drop(subset=["line_id", "delay_min"])


# Exercice 8 : Jointure Silver → Gold

In [42]:
gold = silver_users.join(silver_lines, "line_id", "left")
gold.show(10, truncate=False)


+-------+-------+-------------+---+----------+-------------------+---------+------------+---------+-----------------+-----------------+-----------+----------------+
|line_id|user_id|name         |age|trip_date |actual_duration_min|delay_min|satisfaction|line_name|origin           |destination      |distance_km|avg_duration_min|
+-------+-------+-------------+---+----------+-------------------+---------+------------+---------+-----------------+-----------------+-----------+----------------+
|1      |1      |Alice Dupont |29 |2025-10-01|64                 |2        |null        |IC-01    |Bruxelles-Midi   |Liège-Guillemins |98         |62              |
|1      |1      |Alice Dupont |29 |2025-10-01|64                 |2        |null        |IC-01    |Bruxelles-Midi   |Liège-Guillemins |98         |62              |
|1      |1      |Alice Dupont |29 |2025-10-01|64                 |2        |null        |IC-01    |Bruxelles-Midi   |Liège-Guillemins |98         |62              |
|1      |1

# Exercice 9 : Calculer le retard moyen par ligne

In [45]:
from pyspark.sql.functions import avg

gold_delay = gold.groupBy("line_name").agg(avg("delay_min").alias("avg_delay"))
gold_delay.orderBy(col("avg_delay").desc()).show(10)


+---------+------------------+
|line_name|         avg_delay|
+---------+------------------+
|    IC-05|               5.0|
|    IC-04|              3.25|
|    IC-08|2.6666666666666665|
|     S-13|2.6666666666666665|
|    IC-01|              2.25|
|     S-11|1.6666666666666667|
|     S-12|1.6666666666666667|
|    IC-07|1.6666666666666667|
|     S-10|0.6666666666666666|
|    IC-09|0.6666666666666666|
+---------+------------------+
only showing top 10 rows



# Exercice 10 : Identifier les lignes les plus utilisées

In [47]:
from pyspark.sql.functions import count

usage = gold.groupBy("line_name").agg(count("*").alias("nb_trips"))
usage.orderBy(col("nb_trips").desc()).show()


+---------+--------+
|line_name|nb_trips|
+---------+--------+
|    IC-05|      64|
|    IC-03|      64|
|    IC-04|      64|
|    IC-01|      64|
|    IC-02|      64|
|    IC-06|      48|
|     S-10|      48|
|     S-11|      48|
|     S-12|      48|
|     S-15|      48|
|    IC-07|      48|
|    IC-08|      48|
|    IC-09|      48|
|     S-13|      48|
|     S-14|      48|
+---------+--------+



# Exercice 11 : Satisfaction moyenne par ligne

In [48]:
satisfaction = gold.groupBy("line_name").agg(avg("satisfaction").alias("avg_satisfaction"))
satisfaction.show()


+---------+----------------+
|line_name|avg_satisfaction|
+---------+----------------+
|    IC-06|            null|
|     S-10|            null|
|     S-11|            null|
|    IC-05|            null|
|    IC-03|            null|
|    IC-04|            null|
|     S-12|            null|
|     S-15|            null|
|    IC-01|            null|
|    IC-02|            null|
|    IC-07|            null|
|    IC-08|            null|
|    IC-09|            null|
|     S-13|            null|
|     S-14|            null|
+---------+----------------+



# Exercice 12 : Créer une vue SQL et exécuter des requêtes

In [51]:
# créer departure et arrival (type date) à partir de trip_date et actual_duration_min
from pyspark.sql import functions as F

gold = (
    gold
    .withColumn("departure", F.col("trip_date"))
    .withColumn(
        "arrival",
        F.expr("timestampadd(MINUTE, actual_duration_min, trip_date)")
    )
)

gold.show()

+-------+-------+-------------+---+----------+-------------------+---------+------------+---------+-----------------+-----------------+-----------+----------------+----------+-------------------+
|line_id|user_id|         name|age| trip_date|actual_duration_min|delay_min|satisfaction|line_name|           origin|      destination|distance_km|avg_duration_min| departure|            arrival|
+-------+-------+-------------+---+----------+-------------------+---------+------------+---------+-----------------+-----------------+-----------+----------------+----------+-------------------+
|      1|      1| Alice Dupont| 29|2025-10-01|                 64|        2|        null|    IC-01|   Bruxelles-Midi| Liège-Guillemins|         98|              62|2025-10-01|2025-10-01 01:04:00|
|      1|      1| Alice Dupont| 29|2025-10-01|                 64|        2|        null|    IC-01|   Bruxelles-Midi| Liège-Guillemins|         98|              62|2025-10-01|2025-10-01 01:04:00|
|      1|      1| Al

In [52]:
gold.createOrReplaceTempView("gold_trips")

spark.sql("""
          SELECT line_id, departure, arrival, AVG(delay_min) as avg_delay
          FROM gold_trips
          GROUP BY line_id, departure, arrival
          ORDER BY avg_delay DESC
          """).show()


+-------+----------+-------------------+---------+
|line_id| departure|            arrival|avg_delay|
+-------+----------+-------------------+---------+
|      5|2025-10-15|2025-10-15 02:02:00|      7.0|
|      5|2025-10-03|2025-10-03 02:01:00|      6.0|
|      8|2025-10-17|2025-10-17 01:41:00|      6.0|
|      4|2025-10-22|2025-10-22 01:03:00|      6.0|
|      5|2025-10-09|2025-10-09 02:00:00|      5.0|
|      1|2025-10-08|2025-10-08 01:07:00|      5.0|
|     13|2025-10-12|2025-10-12 00:32:00|      4.0|
|      7|2025-10-10|2025-10-10 00:56:00|      4.0|
|      4|2025-10-09|2025-10-09 01:01:00|      4.0|
|     11|2025-10-12|2025-10-12 00:21:00|      3.0|
|      1|2025-10-20|2025-10-20 01:05:00|      3.0|
|      8|2025-10-04|2025-10-04 01:38:00|      3.0|
|      6|2025-10-16|2025-10-16 01:25:00|      3.0|
|     13|2025-10-19|2025-10-19 00:31:00|      3.0|
|      3|2025-10-02|2025-10-02 00:52:00|      3.0|
|     10|2025-10-05|2025-10-05 00:44:00|      3.0|
|      5|2025-10-22|2025-10-22 

In [53]:
from pyspark.sql.functions import avg, round as spark_round

gold.groupBy("line_id", "departure", "arrival") \
    .agg(spark_round(avg("delay_min"), 2).alias("avg_delay")) \
    .orderBy(col("avg_delay").desc()) \
    .show()


+-------+----------+-------------------+---------+
|line_id| departure|            arrival|avg_delay|
+-------+----------+-------------------+---------+
|      5|2025-10-15|2025-10-15 02:02:00|      7.0|
|      5|2025-10-03|2025-10-03 02:01:00|      6.0|
|      8|2025-10-17|2025-10-17 01:41:00|      6.0|
|      4|2025-10-22|2025-10-22 01:03:00|      6.0|
|      5|2025-10-09|2025-10-09 02:00:00|      5.0|
|      1|2025-10-08|2025-10-08 01:07:00|      5.0|
|     13|2025-10-12|2025-10-12 00:32:00|      4.0|
|      7|2025-10-10|2025-10-10 00:56:00|      4.0|
|      4|2025-10-09|2025-10-09 01:01:00|      4.0|
|     11|2025-10-12|2025-10-12 00:21:00|      3.0|
|      1|2025-10-20|2025-10-20 01:05:00|      3.0|
|      8|2025-10-04|2025-10-04 01:38:00|      3.0|
|      6|2025-10-16|2025-10-16 01:25:00|      3.0|
|     13|2025-10-19|2025-10-19 00:31:00|      3.0|
|      3|2025-10-02|2025-10-02 00:52:00|      3.0|
|     10|2025-10-05|2025-10-05 00:44:00|      3.0|
|      5|2025-10-22|2025-10-22 

# Exercice 13 : Ajouter une colonne de ponctualité

In [54]:
from pyspark.sql.functions import when

gold = gold.withColumn("is_on_time", when(col("delay_min") <= 5, True).otherwise(False))
gold.groupBy("is_on_time").count().show()


+----------+-----+
|is_on_time|count|
+----------+-----+
|      true|  736|
|     false|   64|
+----------+-----+



# Exercice 14 : Identifier les pires lignes (retards fréquents)

In [55]:
from pyspark.sql.functions import col, avg, when

gold.groupBy("line_id") \
    .agg(
    avg(when(col("delay_min") > 5, 1).otherwise(0)).alias("freq_delays")
) \
    .show()


+-------+------------------+
|line_id|       freq_delays|
+-------+------------------+
|      1|               0.0|
|      2|               0.0|
|      5|               0.5|
|     14|               0.0|
|      3|               0.0|
|      4|              0.25|
|      8|0.3333333333333333|
|     12|               0.0|
|      6|               0.0|
|      7|               0.0|
|      9|               0.0|
|     10|               0.0|
|     11|               0.0|
|     13|               0.0|
|     15|               0.0|
+-------+------------------+



# Exercice 15 : Sauvegarde Gold en DB

In [56]:
gold.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres:5432/events") \
    .option("dbtable", "gold_train_stats") \
    .option("user", "pguser") \
    .option("password", "pgpass") \
    .mode("overwrite") \
    .save()

# Exercice 16 : Rechargement depuis DB et requête Spark SQL

In [57]:
gold_db = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres:5432/events") \
    .option("dbtable", "gold_train_stats") \
    .option("user", "pguser") \
    .option("password", "pgpass") \
    .load()

gold_db.show(5)

+-------+-------+------------+---+----------+-------------------+---------+------------+---------+--------------+----------------+-----------+----------------+----------+-------------------+----------+
|line_id|user_id|        name|age| trip_date|actual_duration_min|delay_min|satisfaction|line_name|        origin|     destination|distance_km|avg_duration_min| departure|            arrival|is_on_time|
+-------+-------+------------+---+----------+-------------------+---------+------------+---------+--------------+----------------+-----------+----------------+----------+-------------------+----------+
|      1|      1|Alice Dupont| 29|2025-10-01|                 64|        2|        null|    IC-01|Bruxelles-Midi|Liège-Guillemins|         98|              62|2025-10-01|2025-10-01 01:04:00|      true|
|      1|      1|Alice Dupont| 29|2025-10-01|                 64|        2|        null|    IC-01|Bruxelles-Midi|Liège-Guillemins|         98|              62|2025-10-01|2025-10-01 01:04:00|  

# Exercice 17 : Durée moyenne des trajets

In [59]:
spark.sql("""
          SELECT departure, arrival, AVG(avg_duration_min) AS avg_duration
          FROM gold_trips
          GROUP BY departure, arrival
          ORDER BY avg_duration DESC
          """).show()

+----------+-------------------+------------+
| departure|            arrival|avg_duration|
+----------+-------------------+------------+
|2025-10-09|2025-10-09 02:00:00|       115.0|
|2025-10-22|2025-10-22 01:57:00|       115.0|
|2025-10-03|2025-10-03 02:01:00|       115.0|
|2025-10-15|2025-10-15 02:02:00|       115.0|
|2025-10-04|2025-10-04 01:38:00|        95.0|
|2025-10-11|2025-10-11 01:34:00|        95.0|
|2025-10-17|2025-10-17 01:41:00|        95.0|
|2025-10-03|2025-10-03 01:20:00|        82.0|
|2025-10-10|2025-10-10 01:22:00|        82.0|
|2025-10-16|2025-10-16 01:25:00|        82.0|
|2025-10-08|2025-10-08 01:07:00|        62.0|
|2025-10-01|2025-10-01 01:04:00|        62.0|
|2025-10-14|2025-10-14 01:01:00|        62.0|
|2025-10-20|2025-10-20 01:05:00|        62.0|
|2025-10-22|2025-10-22 01:03:00|        57.0|
|2025-10-09|2025-10-09 01:01:00|        57.0|
|2025-10-02|2025-10-02 00:58:00|        57.0|
|2025-10-15|2025-10-15 00:59:00|        57.0|
|2025-10-08|2025-10-08 00:55:00|  

In [63]:
from pyspark.sql.functions import avg

gold.groupBy("departure", "arrival") \
    .agg(spark_round(avg("avg_duration_min"), 2).alias("avg_duration_min")) \
    .orderBy(col("avg_duration_min").desc()) \
    .show()


+----------+-------------------+----------------+
| departure|            arrival|avg_duration_min|
+----------+-------------------+----------------+
|2025-10-09|2025-10-09 02:00:00|           115.0|
|2025-10-22|2025-10-22 01:57:00|           115.0|
|2025-10-03|2025-10-03 02:01:00|           115.0|
|2025-10-15|2025-10-15 02:02:00|           115.0|
|2025-10-04|2025-10-04 01:38:00|            95.0|
|2025-10-11|2025-10-11 01:34:00|            95.0|
|2025-10-17|2025-10-17 01:41:00|            95.0|
|2025-10-03|2025-10-03 01:20:00|            82.0|
|2025-10-10|2025-10-10 01:22:00|            82.0|
|2025-10-16|2025-10-16 01:25:00|            82.0|
|2025-10-08|2025-10-08 01:07:00|            62.0|
|2025-10-01|2025-10-01 01:04:00|            62.0|
|2025-10-14|2025-10-14 01:01:00|            62.0|
|2025-10-20|2025-10-20 01:05:00|            62.0|
|2025-10-22|2025-10-22 01:03:00|            57.0|
|2025-10-09|2025-10-09 01:01:00|            57.0|
|2025-10-02|2025-10-02 00:58:00|            57.0|


# Exercice 18 — Délai moyen par ville de départ

In [64]:
spark.sql("""
          SELECT departure,
                 ROUND(AVG(delay_min), 2) as avg_delay
          FROM gold_trips
          GROUP BY departure
          ORDER BY avg_delay DESC
          """).show()


+----------+---------+
| departure|avg_delay|
+----------+---------+
|2025-10-22|      4.0|
|2025-10-15|     3.67|
|2025-10-12|      3.0|
|2025-10-08|      2.5|
|2025-10-17|      2.5|
|2025-10-04|      2.5|
|2025-10-19|     2.33|
|2025-10-09|     2.33|
|2025-10-10|      2.0|
|2025-10-02|      2.0|
|2025-10-05|      2.0|
|2025-10-03|     1.67|
|2025-10-16|      1.5|
|2025-10-06|      1.0|
|2025-10-18|      0.5|
|2025-10-13|      0.5|
|2025-10-01|      0.5|
|2025-10-20|      0.5|
|2025-10-14|      0.5|
|2025-10-11|    -0.33|
+----------+---------+
only showing top 20 rows



In [65]:
gold.groupBy("departure") \
    .agg(spark_round(avg("delay_min"), 2).alias("avg_delay")) \
    .orderBy(col("avg_delay").desc()) \
    .show()


+----------+---------+
| departure|avg_delay|
+----------+---------+
|2025-10-22|      4.0|
|2025-10-15|     3.67|
|2025-10-12|      3.0|
|2025-10-08|      2.5|
|2025-10-17|      2.5|
|2025-10-04|      2.5|
|2025-10-19|     2.33|
|2025-10-09|     2.33|
|2025-10-10|      2.0|
|2025-10-02|      2.0|
|2025-10-05|      2.0|
|2025-10-03|     1.67|
|2025-10-16|      1.5|
|2025-10-06|      1.0|
|2025-10-18|      0.5|
|2025-10-13|      0.5|
|2025-10-01|      0.5|
|2025-10-20|      0.5|
|2025-10-14|      0.5|
|2025-10-11|    -0.33|
+----------+---------+
only showing top 20 rows



# Exercice 19 — Taux de satisfaction par destination

In [66]:
spark.sql("""
          SELECT arrival,
                 ROUND(AVG(satisfaction), 2) as avg_satisfaction
          FROM gold_trips
          GROUP BY arrival
          ORDER BY avg_satisfaction DESC
          """).show()


+-------------------+----------------+
|            arrival|avg_satisfaction|
+-------------------+----------------+
|2025-10-01 01:04:00|            null|
|2025-10-02 00:52:00|            null|
|2025-10-03 01:20:00|            null|
|2025-10-03 00:53:00|            null|
|2025-10-10 01:22:00|            null|
|2025-10-11 00:49:00|            null|
|2025-10-12 00:21:00|            null|
|2025-10-12 00:26:00|            null|
|2025-10-15 02:02:00|            null|
|2025-10-16 01:25:00|            null|
|2025-10-17 01:41:00|            null|
|2025-10-18 00:41:00|            null|
|2025-10-19 00:27:00|            null|
|2025-10-20 01:05:00|            null|
|2025-10-21 00:53:00|            null|
|2025-10-02 00:58:00|            null|
|2025-10-05 00:19:00|            null|
|2025-10-07 00:28:00|            null|
|2025-10-13 00:47:00|            null|
|2025-10-14 01:01:00|            null|
+-------------------+----------------+
only showing top 20 rows



In [67]:
gold.groupBy("arrival") \
    .agg(spark_round(avg("satisfaction"), 2).alias("avg_satisfaction")) \
    .orderBy(col("avg_satisfaction").desc()) \
    .show()


+-------------------+----------------+
|            arrival|avg_satisfaction|
+-------------------+----------------+
|2025-10-01 01:04:00|            null|
|2025-10-02 00:52:00|            null|
|2025-10-03 01:20:00|            null|
|2025-10-03 00:53:00|            null|
|2025-10-10 01:22:00|            null|
|2025-10-11 00:49:00|            null|
|2025-10-12 00:21:00|            null|
|2025-10-12 00:26:00|            null|
|2025-10-15 02:02:00|            null|
|2025-10-16 01:25:00|            null|
|2025-10-17 01:41:00|            null|
|2025-10-18 00:41:00|            null|
|2025-10-19 00:27:00|            null|
|2025-10-20 01:05:00|            null|
|2025-10-21 00:53:00|            null|
|2025-10-02 00:58:00|            null|
|2025-10-05 00:19:00|            null|
|2025-10-07 00:28:00|            null|
|2025-10-13 00:47:00|            null|
|2025-10-14 01:01:00|            null|
+-------------------+----------------+
only showing top 20 rows



# Exercice 20 — Export final en base PostgreSQL

In [68]:
from pyspark.sql.functions import count

final_kpi = (
    gold.groupBy("line_id", "departure", "arrival")
    .agg(
        spark_round(avg("delay_min"), 2).alias("avg_delay"),
        spark_round(avg("satisfaction"), 2).alias("avg_satisfaction"),
        count("*").alias("nb_trips")
    )
)

# Écriture en base PostgreSQL
final_kpi.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres:5432/events") \
    .option("dbtable", "gold_train_kpi") \
    .option("user", "pguser") \
    .option("password", "pgpass") \
    .mode("overwrite") \
    .save()

display(final_kpi.limit(10))


DataFrame[line_id: string, departure: date, arrival: timestamp, avg_delay: double, avg_satisfaction: double, nb_trips: bigint]